# APPLIED MACHINE LEARNING ASSIGNMENT 1

 The three models i have chosen for the assignment are logistic regression,random forest and descision tree.

## importing packages

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
with open ("SMSSpamCollection","r")as f:
    raw=f.read()

In [3]:
print(raw[:1000])


ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam

In [4]:
df=pd.read_csv("SMSSpamCollection",delimiter='\t',names=["label","sms"])

In [5]:
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.describe()

,label,sms
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [7]:
df.groupby("label").describe()

sms                                                               
      count unique                                                top freq
label                                                                     
ham    4825   4516                             Sorry, I'll call later   30
spam    747    653  Please call our customer service representativ...    4

In [8]:
df.groupby("label").describe()

sms                                                               
      count unique                                                top freq
label                                                                     
ham    4825   4516                             Sorry, I'll call later   30
spam    747    653  Please call our customer service representativ...    4

In [9]:
df.describe()

,label,sms
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [10]:
from sklearn.model_selection import train_test_split

X = df['sms']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=0.2)


In [11]:
print(len(X_train),len(y_train),len(X_test),len(y_test))

4457 4457 1115 1115


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(X_train)
X_train_text_features = vectorizer.transform(list(X_train))

## logistic regression

Logistic regression is the appropriate regression analysis to conduct when the dependent variable is binary.
Like all regression analyses, the logistic regression is a predictive analysis.  Logistic regression is used to describe data and to explain the relationship between one dependent binary variable and one or more nominal, ordinal, interval or ratio-level independent variables.


In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear',penalty='l1')

model.fit(X_train_text_features, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_train_predicted = model.predict(X_train_text_features)

print("The fraction of correctly classified samples is {}".format(accuracy_score(y_train, y_train_predicted)))
print("The number of correctly classified samples is {}".format(accuracy_score(y_train, 
                                                                               y_train_predicted, normalize=False)))
pd.DataFrame(confusion_matrix( y_train,y_train_predicted),
              index=['true ham', 'true spam'], 
             columns=['pred ham', 'pred spam'])

The fraction of correctly classified samples is 0.9706080323087278
The number of correctly classified samples is 4326


,pred ham,pred spam
true ham,3840,19
true spam,112,486


In [15]:
from sklearn.model_selection import cross_val_score

lr_model = LogisticRegression()
cv_scores = cross_val_score(lr_model, X=X_train_text_features, y=y_train, cv=5, n_jobs=4)
print(cv_scores)

[0.97309417 0.96524664 0.96076233 0.96071829 0.95393258]


In [16]:
X_test_text_features = vectorizer.transform(list(X_test))
y_test_predicted = model.predict(X_test_text_features)



In [17]:
pd.DataFrame(confusion_matrix( y_true=y_test,y_pred=y_test_predicted), 
             index=['true ham', 'true spam'], 
             columns=['pred ham', 'pred spam'])

,pred ham,pred spam
true ham,956,10
true spam,19,130


In [40]:
from sklearn import metrics
recall_test=metrics.recall_score(y_test,model.predict(X_test_text_features),pos_label="spam")

In [41]:
print(recall_test)

0.87248322147651


In [42]:
precision_test=metrics.precision_score(y_test,model.predict(X_test_text_features),pos_label="ham")

In [43]:
print(precision_test)

0.989648033126294


# Random forest classifier

A random forest is an estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement.
Random decision forests correct for decision trees' habit of overfitting to their training set.

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
model1 = RandomForestClassifier()

model1.fit(X_train_text_features, y_train)

C:\Users\krishna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
y_train_predicted1 = model1.predict(X_train_text_features)

print("The fraction of correctly classified samples is {}".format(accuracy_score(y_train, y_train_predicted1)))
print("The number of correctly classified samples is {}".format(accuracy_score(y_train, 
                                                                               y_train_predicted1, normalize=False)))
pd.DataFrame(confusion_matrix( y_train,y_train_predicted1),
              index=['true ham', 'true spam'], 
             columns=['pred ham', 'pred spam'])

The fraction of correctly classified samples is 0.99798070450976
The number of correctly classified samples is 4448


,pred ham,pred spam
true ham,3859,0
true spam,9,589


In [21]:


rf_model = RandomForestClassifier()
cv_scores1 = cross_val_score(rf_model, X=X_train_text_features, y=y_train, cv=5, n_jobs=4)
print(cv_scores1)

[0.97421525 0.96973094 0.95627803 0.9708193  0.96741573]


In [22]:
y_test_predicted1 = model1.predict(X_test_text_features)

In [23]:
pd.DataFrame(confusion_matrix( y_true=y_test,y_pred=y_test_predicted1), 
             index=['true ham', 'true spam'], 
             columns=['pred ham', 'pred spam'])

,pred ham,pred spam
true ham,965,1
true spam,19,130


In [36]:
from sklearn import metrics
recall_test1=metrics.recall_score(y_test,model1.predict(X_test_text_features),pos_label="spam")

In [37]:
print(recall_test2)

0.8523489932885906


In [38]:
precision_test1=metrics.recall_score(y_test,model1.predict(X_test_text_features),pos_label="ham")

In [39]:
print(precision_test1)

0.9989648033126294


# Decision Tree Classifier

Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.
 decision trees learn from data to approximate a sine curve with a set of if-then-else decision rules. The deeper the tree, the more complex the decision rules and the fitter the model.

DecisionTreeClassifier is capable of both binary  classification and multiclass  classification.

In [24]:
from sklearn.tree import DecisionTreeClassifier


In [25]:
model2 = DecisionTreeClassifier()

model2.fit(X_train_text_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [26]:
y_train_predicted2 = model2.predict(X_train_text_features)

print("The fraction of correctly classified samples is {}".format(accuracy_score(y_train, y_train_predicted2)))
print("The number of correctly classified samples is {}".format(accuracy_score(y_train, 
                                                                               y_train_predicted1, normalize=False)))
pd.DataFrame(confusion_matrix( y_train,y_train_predicted2),
              index=['true ham', 'true spam'], 
             columns=['pred ham', 'pred spam'])

The fraction of correctly classified samples is 0.9997756338344178
The number of correctly classified samples is 4448


,pred ham,pred spam
true ham,3859,0
true spam,1,597


In [27]:
dt_model = DecisionTreeClassifier()
cv_scores2 = cross_val_score(rf_model, X=X_train_text_features, y=y_train, cv=5, n_jobs=4)
print(cv_scores2)

[0.97982063 0.96860987 0.96636771 0.96857464 0.96404494]


In [28]:
y_test_predicted2 = model2.predict(X_test_text_features)

In [29]:
pd.DataFrame(confusion_matrix( y_true=y_test,y_pred=y_test_predicted2), 
             index=['true ham', 'true spam'], 
             columns=['pred ham', 'pred spam'])

,pred ham,pred spam
true ham,953,13
true spam,22,127


In [34]:
from sklearn import metrics
recall_test2=metrics.recall_score(y_test,model2.predict(X_test_text_features),pos_label="spam")

In [35]:
print(recall_test2)

0.8523489932885906


In [32]:
precision_test2=metrics.recall_score(y_test,model2.predict(X_test_text_features),pos_label="ham")

In [33]:
print(precision_test2)

0.9865424430641822
